Code for Measuring Model Computation Time

Single and 5-model ensemble compared

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import tensorflow.keras.layers as tfl
from sklearn.preprocessing import StandardScaler, normalize
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LambdaCallback
from tensorflow.keras.models import Sequential, load_model, Model
from scipy.signal import find_peaks
import time

In [26]:
#Recieves data and applies z-score standardisation on all channels
def standardise(stored_data):
    scaler = StandardScaler()
    standard_stored_data = scaler.fit_transform(stored_data)
    return standard_stored_data

In [27]:
#Following code was made by adapting original code by Wen et. al (2021)
#Provided in their paper: "A convolutional neural network to identify motor units
#from high-density surface electromyography signals inreal time"
#Original code can be found here: https://github.com/ywen3/dcnn_mu_decomp/blob/main/hdEMG_DCNN.ipynb

#Used for calculating RoA during model training

def RoA_m(y_true, y_pred):
    threshold = 3*tf.math.reduce_std(y_pred)
    y_pred_binary = tf.where(y_pred>=threshold, 1., 0.)
    y_comp = y_pred_binary + y_true
    true_positives = tf.shape(tf.where(y_comp == 2))[0]
    unmatched = tf.shape(tf.where(y_comp == 1))[0]
    return true_positives/(true_positives + unmatched)


class AccuracyCallback(Callback):
    def __init__(self, metric_name = 'accuracy'):
        super().__init__()
        self.metric_name = metric_name
        self.val_metric = []
        self.metric = []
        self.val_metric_mean = 0
        self.metric_mean = 0
        self.best_metric = 0
        
    def on_epoch_end(self, epoch, logs=None):
#         print('Accuracycallback')
        # extract values from logs
        self.val_metric = []
        self.metric = []
        for log_name, log_value in logs.items():
            if log_name.find(self.metric_name) != -1:
                if log_name.find('val') != -1:
                    self.val_metric.append(log_value)
                else:
                    self.metric.append(log_value)

        self.val_metric_mean = np.mean(self.val_metric)
        self.metric_mean = np.mean(self.metric)
        logs['val_{}'.format(self.metric_name)] = np.mean(self.val_metric)   # replace it with your metrics
        logs['{}'.format(self.metric_name)] = np.mean(self.metric)   # replace it with your metrics

In [28]:
#Window incloming signal and fit a scaler to transform data online
def windowtime(EMGtrain, spiketrain, window_size):
    scaler = StandardScaler()
    scaler.fit(EMGtrain)
    
    x_train = []
    y_train = []
    for i in range(30,EMGtrain.shape[0]-120):
        x_train.append(EMGtrain[i-10:i+(window_size-10),:])
        y_train.append(spiketrain[i, 0:5])
    
    return np.array(x_train), np.array(y_train), scaler

In [29]:
#Provides the predictoins of the ensmble with naive weighting
def naiveEnsemblePredict(output):
    y_pred = tf.math.add_n(output)
    y_pred = tf.squeeze(y_pred,2)
    return y_pred

In [30]:
#Generates the ensemble model by combining multiple models
def ensemble_model_gen(input_shape, names):
    input_signal = tf.keras.Input(shape = input_shape)
    out = []
    
    for i in names:
        output = load_model(i, custom_objects={"RoA_m": RoA_m})(input_signal)
        out.append(output)
    
    model = tf.keras.Model(inputs = input_signal, outputs = out)
    return model

In [31]:
#Gets 1 second from a 30 dB, 20 MU, no lowpassing, no shift signal and extracts windows and scaler
var = 'noise'
noise = '30dB'
fold = 1
EMGtest = np.load('{}_data/{}_fold{}_x.npy'.format(var, noise, fold))[0:2048]
spikes = np.load('{}_data/{}_fold{}_y.npy'.format(var, noise, fold))[0:2048]
X , Y, scaler = windowtime(EMGtest, spikes, 60)

In [32]:
#Get single model
conv_model = load_model('{}_models/bestvR_big_fold{}.h5'.format(var, fold), custom_objects={"RoA_m": RoA_m})

Computation time and decompostion rate is measured for both models

Time to standardise incoming windows and decompose window batch is included in the prediction time

In [44]:
comp_time = []
spike = []
stack = 42 #batch of windows to be decomposed at once
prepare = conv_model(X[0:stack])
for k  in range(0, X.shape[0]-11,stack):
    EMG = X[k:(k+stack)]
    start_time = time.time()
    #Standardise incoming signal
    for i in range(EMG.shape[0]):
        EMG[i] = scaler.transform(EMG[i])
    predictions = conv_model(EMG)
    comp_time.append(time.time() - start_time)
    spike.append(tf.squeeze(predictions))
print('Computation time: '+ str((np.mean(comp_time))*1000) + ' ms')
print('Decomposition rate (needs to be >2048): ' + str(1/(np.mean(comp_time)/stack)))

Computation time: 19.86106236775716 ms
Decomposition rate (needs to be >2048): 2114.6905045816493


In [16]:
noises = ['20dB','15dB','10dB','5dB','0dB']
names = []
for n in noises:
    names.append('{}_models/bestvR_{}_fold{}.h5'.format(var, n, fold))
ensemble_model = ensemble_model_gen((60, 192), names)

In [62]:
comp_time = []
spike = []
stack = 192 #batch of windows to be decomposed at once
prepare = ensemble_model(X[0:stack])
for k in range(0, X.shape[0]-11,stack):
    EMG = X[k:(k+stack)]
    start_time = time.time()
    #Standardise incoming signal
    for i in range(EMG.shape[0]):
        EMG[i] = scaler.transform(EMG[i])
    predictions = naiveEnsemblePredict(ensemble_model(EMG))
    comp_time.append(time.time() - start_time)
    spike.append(tf.squeeze(predictions))
print('Computation time: '+ str((np.mean(comp_time))*1000) + ' ms')
print('Decomposition rate (needs to be >2048): ' + str(1/(np.mean(comp_time)/stack)))

Computation time: 90.75460433959961 ms
Decomposition rate (needs to be >2048): 2115.5951413940907
